In [1]:
import re
import string

import numpy as np
import pandas as pd

from unidecode import unidecode

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Exemplo
example = 'Vamos entender os 3 passos de pré-processamento de texto de forma rápida! [ç, ã, ê, ü, $, ö]'
print(example)

Vamos entender os 3 passos de pré-processamento de texto de forma rápida! [ç, ã, ê, ü, $, ö]


In [3]:
# Lowercase

example = example.lower()
print(example)

vamos entender os 3 passos de pré-processamento de texto de forma rápida! [ç, ã, ê, ü, $, ö]


In [4]:
# Remoção de pontuação e símbolos

# Tabela de pontuação
punctuation = string.punctuation
# Criar tabela de tradução que susbstituirá toda ponutação por um espaço em branco
trantab = str.maketrans(punctuation, len(punctuation)*' ')
# Traduzir nosso exemplo
example = example.translate(trantab)

print(example)


vamos entender os 3 passos de pré processamento de texto de forma rápida   ç  ã  ê  ü     ö 


In [5]:
# Remover caracteres especiais (acentos e afins)

# Método 1
def remove_special_chars(input_text):
    input_text = re.sub(u'[áãâà]', 'a', input_text)
    input_text = re.sub(u'[éèê]', 'e', input_text)
    input_text = re.sub(u'[íì]', 'i', input_text)
    input_text = re.sub(u'[óõôò]', 'o', input_text)
    input_text = re.sub(u'[úùü]', 'u', input_text)
    input_text = re.sub(u'[ç]', 'c', input_text)
    return input_text

print(remove_special_chars(example))


# Método 2
example = unidecode(example)
print(example)


vamos entender os 3 passos de pre processamento de texto de forma rapida   c  a  e  u     ö 
vamos entender os 3 passos de pre processamento de texto de forma rapida   c  a  e  u     o 


In [6]:
# Remover dígitos

def remove_digits(input_text):
    import re
    return re.sub('\d+', '', input_text)

example = remove_digits(example)
print(example)


vamos entender os  passos de pre processamento de texto de forma rapida   c  a  e  u     o 


In [7]:
# Remover stopwords

try:
    stopwords_list = stopwords.words('portuguese')
except:
    nltk.download('stopwords')
    stopwords_list = stopwords.words('portuguese')

def remove_stopwords(input_text, stopwords_list):
    words = input_text.split()
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 1]
    return " ".join(clean_words)

example = remove_stopwords(example, stopwords_list)
print(example)

vamos entender passos pre processamento texto forma rapida


In [8]:
# Radicalização

def stemming(input_text):
    porter = PorterStemmer()
    words = input_text.split()
    stemmed_words = [porter.stem(word) for word in words]
    return " ".join(stemmed_words)
example = stemming(example)
print(example)

vamo entend passo pre processamento texto forma rapida


In [9]:
# O TF-IDF espera um iterable como corpus, logo nosso exemplo precisa se tornar um array ou lista

example = np.array(
    [
        'vamo entend passo pre processamento texto forma rapida vamo',
        'vamo pular psicina'
    ]
)
example

array(['vamo entend passo pre processamento texto forma rapida vamo',
       'vamo pular psicina'], dtype='<U59')

In [50]:
# O TF-IDF

# Instanciar o vetorizador do sklearn
tfv = TfidfVectorizer()
# Ajustar os pesos e fazer a transformação do corpus para a representação TF-IDF
tfidf = tfv.fit_transform(example)


tfidf = pd.DataFrame.sparse.from_spmatrix(tfidf, columns=tfv.get_feature_names())
print(tfidf)


     entend     forma     passo       pre  processamento   psicina     pular  \
0  0.332872  0.332872  0.332872  0.332872       0.332872  0.000000  0.000000   
1  0.000000  0.000000  0.000000  0.000000       0.000000  0.631667  0.631667   

     rapida     texto      vamo  
0  0.332872  0.332872  0.473682  
1  0.000000  0.000000  0.449436  


In [58]:
#IMPORTANTE
# O TfidfVectorizer() se ajusta ao corpus, logo, deve ser ajustado nos dados de treino e no momento de codificar os dados de teste ou na inferência
# usar apenas a chamada tfv.transform(test)
